In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import json
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from transformers import AutoTokenizer

#tokenizer= AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

datas = []
path = "../tsv/first-match-scatter/42/test.json"
with open(path) as f:
    works = json.load(f)
for work in tqdm(works[:100]):
    label = work["label"]
    contents = [c["paragraph"] for c in work["contents"]]
    if len(contents) == 5:
        for c in contents:
            datas.append({'label': label, 'text': c})
df = pd.DataFrame.from_dict(datas)
ds = Dataset.from_pandas(df)

100%|█████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 342392.16it/s]


In [4]:
tokenizer= AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

In [5]:
def preprocess_func(exp):
    MAX_LENGTH = 512
    return tokenizer(exp['text'])

tkn_ds = ds.map(preprocess_func, shuffule=False)

Parameter 'function'=<function preprocess_func at 0x7fcb4139ddc0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1 [00:00<?, ?ba/s]

In [6]:
ds

Dataset({
    features: ['label', 'text'],
    num_rows: 490
})

In [7]:
tkn_ds

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 490
})

In [8]:
tkn_ds = tkn_ds.train_test_split(test_size=0.5, shuffle=False)
tkn_ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 245
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 245
    })
})

In [9]:
import torch
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    label_lst = []
    for i, l in enumerate(labels):
        if i % 5 == 0:
            label_lst.append(l)
    #print('pred ', pred.predictions)
    preds = pred.predictions.argmax(-1)
    print('label ', label_lst)
    print('argmax ', preds)
    f1 = f1_score(label_lst, preds, average='weighted')
    acc = accuracy_score(label_lst, preds)
    return {'acc': acc, 'f1': f1}
    

In [19]:
t = torch.Tensor([[1.1, 1.2], [0.9, 0.8]])
print(t)
torch.stack([t])
torch.Tensor([sum(it)/len(t) for it in zip(*t)])

tensor([[1.1000, 1.2000],
        [0.9000, 0.8000]])


tensor([1., 1.])

In [43]:
import torch
from torch import nn
import torch.nn.functional as F
from transformers import BertPreTrainedModel, BertModel
from transformers.modeling_outputs import SequenceClassifierOutput

class ModifiedBERT(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.linear = nn.Linear(in_features=768, out_features=2)

        
    def forward(self, input_ids, attention_mask=None, token_type_ids=None, labels=None, **kwargs):
        outputs = self.bert(
            input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids,
            **kwargs
        )
        
        pooled_output = outputs.pooler_output
        avg_output = torch.Tensor([sum(it)/len(pooled_output) for it in zip(*pooled_output)])#.to('cuda')
        #fst_output = pooled_output[:1] #GRAD Problem?
        logits = self.linear(avg_output)
        logits = torch.stack([logits])#.to('cuda')
        
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, 2), labels[:1].view(-1))
    
        return SequenceClassifierOutput(
            loss = loss,
            logits = logits,
            hidden_states = outputs.hidden_states,
            attentions = outputs.attentions
        )  

In [44]:
del(trainer)
del(modBERT)

In [45]:
torch.cuda.empty_cache()

In [46]:
modBERT = ModifiedBERT.from_pretrained("cl-tohoku/bert-base-japanese-whole-word-masking", num_labels=2)
modBERT.gradient_checkpointing_enable()

loading configuration file https://huggingface.co/cl-tohoku/bert-base-japanese-whole-word-masking/resolve/main/config.json from cache at /home/realive333/.cache/huggingface/transformers/573af37b6c39d672f2df687c06ad7d556476cbe43e5bf7771097187c45a3e7bf.abeb707b5d79387dd462e8bfb724637d856e98434b6931c769b8716c6f287258
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "tokenizer_class": "BertJapaneseTokenizer",
  "transformers_version": "4.20.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 32000
}

loading weights file https://huggingface.co/cl-tohok

In [47]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=1,
    learning_rate=2e-5,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    num_train_epochs=5,
    weight_decay=0.01,
    no_cuda=True
)

trainer = Trainer(
    model=modBERT,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=tkn_ds['train'],
    eval_dataset=tkn_ds['test'],
    tokenizer=tokenizer
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 5


In [36]:
#GradCheck
trainer.train()

The following columns in the training set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
/home/realive333/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 245
  Num Epochs = 5
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 5
  Gradient Accumulation steps = 1
  Total optimization steps = 245


Epoch,Training Loss,Validation Loss,Acc,F1
1,0.626600,0.717888,0.530612,0.367891
2,0.691400,0.728942,0.530612,0.367891
3,0.664200,0.740558,0.530612,0.367891
4,0.701300,0.745562,0.530612,0.367891
5,0.631400,0.748996,0.530612,0.367891


The following columns in the evaluation set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 5


label  [1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1]
argmax  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]


Saving model checkpoint to ./results/checkpoint-49
Configuration saved in ./results/checkpoint-49/config.json
Model weights saved in ./results/checkpoint-49/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-49/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-49/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-245] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 5


...

label  [1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1]
argmax  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]


Saving model checkpoint to ./results/checkpoint-196
Configuration saved in ./results/checkpoint-196/config.json
Model weights saved in ./results/checkpoint-196/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-196/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-196/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-147] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 5


label  [1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1]
argmax  [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1]


Saving model checkpoint to ./results/checkpoint-245
Configuration saved in ./results/checkpoint-245/config.json
Model weights saved in ./results/checkpoint-245/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-245/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-245/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-196] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=245, training_loss=0.6629828861781529, metrics={'train_runtime': 318.5742, 'train_samples_per_second': 3.845, 'train_steps_per_second': 0.769, 'total_flos': 235743908381100.0, 'train_loss': 0.6629828861781529, 'epoch': 5.0})

In [77]:
#AVG
trainer.train()

The following columns in the training set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
/home/realive333/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 245
  Num Epochs = 5
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 5
  Gradient Accumulation steps = 1
  Total optimization steps = 245


Epoch,Training Loss,Validation Loss,Acc,F1
1,0.626600,0.727006,0.530612,0.367891
2,0.688200,0.741066,0.530612,0.367891
3,0.665700,0.755144,0.530612,0.367891
4,0.708500,0.761059,0.530612,0.367891
5,0.633900,0.765094,0.530612,0.367891


The following columns in the evaluation set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 5


pred  [[-6.9388770e-02  6.4100641e-01]
 [-2.2442980e-02  7.3101753e-01]
 [-3.4977030e-02  6.5068680e-01]
 [-3.9835390e-02  5.7460850e-01]
 [-8.1716716e-02  6.9404906e-01]
 [ 1.8193439e-02  7.7643824e-01]
 [-1.9247716e-02  6.9308621e-01]
 [-4.4550210e-02  7.1906894e-01]
 [-9.7597569e-02  7.1656299e-01]
 [ 3.6344919e-02  6.5273386e-01]
 [-1.5169533e-02  6.8064553e-01]
 [-4.5526754e-02  7.1504098e-01]
 [-6.9588348e-02  6.4131159e-01]
 [ 5.0161313e-04  6.6082317e-01]
 [ 8.0156623e-04  6.8944812e-01]
 [-8.4391162e-02  6.3366747e-01]
 [-2.2828678e-02  6.7934316e-01]
 [-2.7450530e-02  6.7068416e-01]
 [ 7.9279840e-03  5.8003873e-01]
 [-6.9758552e-03  6.5892792e-01]
 [-3.3486581e-03  6.4205950e-01]
 [-6.9733538e-02  6.8914914e-01]
 [-9.8602409e-03  7.1465939e-01]
 [-5.5645974e-03  6.6755331e-01]
 [-5.6362372e-02  6.4984572e-01]
 [-1.5815606e-02  6.7229533e-01]
 [-6.5112136e-02  6.4797634e-01]
 [ 1.6276525e-02  8.0803680e-01]
 [-3.5678089e-02  7.0336264e-01]
 [-2.3029074e-02  7.1832484e-01]
 [-1

Saving model checkpoint to ./results/checkpoint-49
Configuration saved in ./results/checkpoint-49/config.json
Model weights saved in ./results/checkpoint-49/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-49/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-49/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-245] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 5


...

pred  [[-0.19609343  0.7677027 ]
 [-0.16168702  0.87025136]
 [-0.15718952  0.7728902 ]
 [-0.18030107  0.7150664 ]
 [-0.21276161  0.82508487]
 [-0.10328841  0.8979084 ]
 [-0.13357425  0.80740273]
 [-0.12923445  0.8037432 ]
 [-0.22221115  0.8411674 ]
 [-0.04642743  0.7354958 ]
 [-0.11176607  0.7772322 ]
 [-0.16268672  0.83219117]
 [-0.16133864  0.73305345]
 [-0.12898785  0.79030305]
 [-0.11419782  0.8044374 ]
 [-0.20646094  0.7557294 ]
 [-0.12801632  0.7845212 ]
 [-0.16007484  0.803299  ]
 [-0.09658699  0.6845448 ]
 [-0.12027368  0.7722162 ]
 [-0.08815157  0.72685295]
 [-0.18771805  0.8071246 ]
 [-0.13721193  0.84200066]
 [-0.12140575  0.78338474]
 [-0.18335432  0.7768288 ]
 [-0.14692472  0.80339485]
 [-0.18046333  0.763319  ]
 [-0.10532854  0.92962956]
 [-0.16985635  0.83753127]
 [-0.14505488  0.8403404 ]
 [-0.1545169   0.82181257]
 [-0.25296536  0.66162014]
 [-0.09647144  0.8857994 ]
 [-0.20553336  0.8509813 ]
 [-0.1684387   0.7842635 ]
 [-0.15739729  0.7397654 ]
 [-0.14315757  0.82843

Saving model checkpoint to ./results/checkpoint-196
Configuration saved in ./results/checkpoint-196/config.json
Model weights saved in ./results/checkpoint-196/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-196/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-196/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-147] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 5


pred  [[-0.2086666   0.7802757 ]
 [-0.17553268  0.8840966 ]
 [-0.16942474  0.78512526]
 [-0.19441803  0.7291831 ]
 [-0.22584035  0.83816355]
 [-0.11538147  0.91000134]
 [-0.14489725  0.81872576]
 [-0.13754462  0.8120533 ]
 [-0.23458745  0.8535435 ]
 [-0.05452862  0.74359715]
 [-0.1212713   0.7867374 ]
 [-0.17427887  0.8437831 ]
 [-0.17039323  0.74210787]
 [-0.14196497  0.80328   ]
 [-0.12553738  0.8157768 ]
 [-0.21857747  0.7678457 ]
 [-0.13840406  0.7949088 ]
 [-0.1733247   0.81654876]
 [-0.10694744  0.6949053 ]
 [-0.13155092  0.7834932 ]
 [-0.09649088  0.73519224]
 [-0.19939315  0.81879956]
 [-0.14988907  0.85467774]
 [-0.13286017  0.79483896]
 [-0.19599557  0.78947   ]
 [-0.1600238   0.81649375]
 [-0.191869    0.7747245 ]
 [-0.11734695  0.9416478 ]
 [-0.18320294  0.8508775 ]
 [-0.15720819  0.8524935 ]
 [-0.16816185  0.8354574 ]
 [-0.26572675  0.6743814 ]
 [-0.110011    0.89933866]
 [-0.22060063  0.86604846]
 [-0.18098503  0.7968095 ]
 [-0.16832562  0.7506935 ]
 [-0.15565477  0.84093

Saving model checkpoint to ./results/checkpoint-245
Configuration saved in ./results/checkpoint-245/config.json
Model weights saved in ./results/checkpoint-245/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-245/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-245/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-196] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=245, training_loss=0.664586133373027, metrics={'train_runtime': 459.0115, 'train_samples_per_second': 2.669, 'train_steps_per_second': 0.534, 'total_flos': 235743908381100.0, 'train_loss': 0.664586133373027, 'epoch': 5.0})

In [88]:
#FST
trainer.train()

The following columns in the training set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
/home/realive333/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 245
  Num Epochs = 5
  Instantaneous batch size per device = 5
  Total train batch size (w. parallel, distributed & accumulation) = 5
  Gradient Accumulation steps = 1
  Total optimization steps = 245


Epoch,Training Loss,Validation Loss,Acc,F1
1,1.058200,2.107628,0.530612,0.367891
2,1.457400,1.733767,0.530612,0.367891
3,1.299600,1.258460,0.530612,0.367891
4,0.707500,1.129556,0.591837,0.491540
5,0.615700,1.298174,0.571429,0.477979


The following columns in the evaluation set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 5


pred  [[[-2.3716033  2.1354716]]

 [[-2.3509037  2.100338 ]]

 [[-2.4399974  1.9390093]]

 [[-2.2239444  1.8624947]]

 [[-2.3785732  1.9671493]]

 [[-2.3839333  2.1073654]]

 [[-2.4366436  2.0288954]]

 [[-2.269755   2.0865319]]

 [[-2.4125981  2.106443 ]]

 [[-2.318344   2.2215807]]

 [[-2.2574527  2.0549753]]

 [[-2.4637544  2.2275357]]

 [[-2.4447176  2.3312361]]

 [[-2.4181051  1.9812515]]

 [[-2.4776201  2.0218952]]

 [[-2.4927278  2.2192175]]

 [[-2.4291556  2.2706535]]

 [[-2.5534153  2.1926243]]

 [[-2.1821866  2.0939658]]

 [[-2.4471133  2.1818438]]

 [[-2.3436823  2.3020616]]

 [[-2.4338884  2.2177682]]

 [[-2.5785956  2.2782629]]

 [[-2.5923574  2.1275287]]

 [[-2.2913082  2.0177705]]

 [[-2.4985158  1.916581 ]]

 [[-2.4673035  2.1015847]]

 [[-2.5702417  1.9283019]]

 [[-2.4663908  1.9838451]]

 [[-2.4658618  2.0187933]]

 [[-2.257643   1.841104 ]]

 [[-2.4501889  2.1007514]]

 [[-2.4991167  2.1575353]]

 [[-2.4866083  2.1011503]]

 [[-2.5641508  2.2560067]]

 [[-2.5049186 

Saving model checkpoint to ./results/checkpoint-49
Configuration saved in ./results/checkpoint-49/config.json
Model weights saved in ./results/checkpoint-49/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-49/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-49/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-245] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 5


...

pred  [[[-0.45936206  1.2468518 ]]

 [[-2.361431    2.4821763 ]]

 [[-0.4715642   0.7157195 ]]

 [[-1.1951411   1.486008  ]]

 [[-0.867842    1.3353944 ]]

 [[ 0.18747088  0.5691981 ]]

 [[-2.9987555   2.441946  ]]

 [[ 0.4908315   0.41150624]]

 [[-0.93860334  1.5091603 ]]

 [[-1.0932239   1.5986043 ]]

 [[ 0.7284453   0.13917783]]

 [[ 0.23066649  0.75557196]]

 [[ 0.3926757   0.5306993 ]]

 [[-2.2432873   1.9463077 ]]

 [[-2.9357932   2.4301808 ]]

 [[-1.1992584   1.770308  ]]

 [[-1.2712986   1.7806726 ]]

 [[-2.625211    2.1684427 ]]

 [[ 0.10894539  0.6794794 ]]

 [[-0.7244876   1.144224  ]]

 [[-0.38554025  1.1089963 ]]

 [[-0.74549353  1.3449165 ]]

 [[-2.8523319   2.6387436 ]]

 [[-2.7795053   2.3038318 ]]

 [[ 0.2959652   0.6481688 ]]

 [[-2.8451693   2.097174  ]]

 [[-2.957958    2.4755135 ]]

 [[-2.6522696   1.6404594 ]]

 [[-2.7780192   2.2900112 ]]

 [[-2.6191337   2.0502067 ]]

 [[-1.5615761   1.8337206 ]]

 [[-2.7278695   2.5523293 ]]

 [[-2.5882473   2.3586953 ]]

 [[-

Saving model checkpoint to ./results/checkpoint-196
Configuration saved in ./results/checkpoint-196/config.json
Model weights saved in ./results/checkpoint-196/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-196/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-196/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-147] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 245
  Batch size = 5


pred  [[[-0.9654656   1.586047  ]]

 [[-2.6719897   2.681312  ]]

 [[-0.27873135  0.57681435]]

 [[-1.5869175   1.7777798 ]]

 [[-1.1457483   1.6406976 ]]

 [[ 0.4930501   0.3451546 ]]

 [[-3.1117184   2.6360445 ]]

 [[ 0.49652183  0.41554624]]

 [[-1.8427587   2.1527815 ]]

 [[-0.72694355  1.3413303 ]]

 [[ 0.8477582   0.01229865]]

 [[ 0.17757276  0.7766961 ]]

 [[ 0.36204544  0.58053756]]

 [[-2.7000544   2.338081  ]]

 [[-3.1247942   2.5879323 ]]

 [[-2.0532246   2.3353536 ]]

 [[-1.7524639   2.1463945 ]]

 [[-2.9910452   2.409278  ]]

 [[ 0.26487097  0.5305051 ]]

 [[-0.40336984  0.84168285]]

 [[-0.972952    1.5587095 ]]

 [[-1.5018792   1.8692162 ]]

 [[-3.1157506   2.8443966 ]]

 [[-3.1224372   2.5517008 ]]

 [[ 0.15386143  0.80476195]]

 [[-3.0406613   2.3903909 ]]

 [[-3.1021419   2.650737  ]]

 [[-2.898799    2.0287611 ]]

 [[-2.9846838   2.5094419 ]]

 [[-2.9286284   2.3570833 ]]

 [[-1.9853839   2.1875448 ]]

 [[-3.0147514   2.7345138 ]]

 [[-2.9081311   2.6385708 ]]

 [[-

Saving model checkpoint to ./results/checkpoint-245
Configuration saved in ./results/checkpoint-245/config.json
Model weights saved in ./results/checkpoint-245/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-245/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-245/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-196] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=245, training_loss=1.0276867068543727, metrics={'train_runtime': 886.6431, 'train_samples_per_second': 1.382, 'train_steps_per_second': 0.276, 'total_flos': 235743908381100.0, 'train_loss': 1.0276867068543727, 'epoch': 5.0})

In [57]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tkn_ds_s['train'],
    batch_size = 1,
    shuffle = False
)


In [58]:
tkn_ds_s

DatasetDict({
    train: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 245
    })
    test: Dataset({
        features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 245
    })
})

In [59]:
len(tkn_ds_s['train'])

245

## TEST MODIFIED BERT

In [81]:
from sklearn.metrics import classification_report

pred_result = trainer.predict(tkn_ds['test'], ignore_keys=['loss', 'last_hidden_state', 'hidden_states', 'attentions'])
result = pred_result.predictions

labels = tkn_ds['test']['label']
label_lst = []
for i, l in enumerate(labels):
    if i % 5 == 0:
        label_lst.append(l)


pred_label= pred_result.predictions.argmax(axis=1).tolist()

print(classification_report(label_lst, pred_label, target_names=['False', 'True']))

The following columns in the test set don't have a corresponding argument in `ModifiedBERT.forward` and have been ignored: text. If text are not expected by `ModifiedBERT.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 245
  Batch size = 5


pred  [[-0.2086666   0.7802757 ]
 [-0.17553268  0.8840966 ]
 [-0.16942474  0.78512526]
 [-0.19441803  0.7291831 ]
 [-0.22584035  0.83816355]
 [-0.11538147  0.91000134]
 [-0.14489725  0.81872576]
 [-0.13754462  0.8120533 ]
 [-0.23458745  0.8535435 ]
 [-0.05452862  0.74359715]
 [-0.1212713   0.7867374 ]
 [-0.17427887  0.8437831 ]
 [-0.17039323  0.74210787]
 [-0.14196497  0.80328   ]
 [-0.12553738  0.8157768 ]
 [-0.21857747  0.7678457 ]
 [-0.13840406  0.7949088 ]
 [-0.1733247   0.81654876]
 [-0.10694744  0.6949053 ]
 [-0.13155092  0.7834932 ]
 [-0.09649088  0.73519224]
 [-0.19939315  0.81879956]
 [-0.14988907  0.85467774]
 [-0.13286017  0.79483896]
 [-0.19599557  0.78947   ]
 [-0.1600238   0.81649375]
 [-0.191869    0.7747245 ]
 [-0.11734695  0.9416478 ]
 [-0.18320294  0.8508775 ]
 [-0.15720819  0.8524935 ]
 [-0.16816185  0.8354574 ]
 [-0.26572675  0.6743814 ]
 [-0.110011    0.89933866]
 [-0.22060063  0.86604846]
 [-0.18098503  0.7968095 ]
 [-0.16832562  0.7506935 ]
 [-0.15565477  0.84093

/home/realive333/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/realive333/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/realive333/.pyenv/versions/anaconda3-2022.05/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  